In [513]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi

def voronoi_finite_polygons_2d(vor, radius=None):
    """
    Reconstruct infinite voronoi regions in a 2D diagram to finite
    regions.

    Parameters
    ----------
    vor : Voronoi
        Input diagram
    radius : float, optional
        Distance to 'points at infinity'.

    Returns
    -------
    regions : list of tuples
        Indices of vertices in each revised Voronoi regions.
    vertices : list of tuples
        Coordinates for revised Voronoi vertices. Same as coordinates
        of input vertices, with 'points at infinity' appended to the
        end.

    """

#     if vor.points.shape[1] != 2:
#         raise ValueError("Requires 2D input")

    new_regions = []
    new_vertices = vor.vertices.tolist()
    point_regions = []

    center = vor.points.mean(axis=0)
    if radius is None:
        radius = vor.points.ptp().max()*2

    # Construct a map containing all ridges for a given point
    all_ridges = {}
    for (p1, p2), (v1, v2) in zip(vor.ridge_points, vor.ridge_vertices):
        all_ridges.setdefault(p1, []).append((p2, v1, v2))
        all_ridges.setdefault(p2, []).append((p1, v1, v2))

    # Reconstruct infinite regions
    for p1, region in enumerate(vor.point_region):
        vertices = vor.regions[region]
#         print(region)

        if all([v >= 0 for v in vertices]):
            # finite region
            new_regions.append(vertices)
            continue

        # reconstruct a non-finite region
        ridges = all_ridges[p1]
        new_region = [v for v in vertices if v >= 0]

        for p2, v1, v2 in ridges:
            if v2 < 0:
                v1, v2 = v2, v1
            if v1 >= 0:
                # finite ridge: already in the region
                continue

            # Compute the missing endpoint of an infinite ridge

            t = vor.points[p2] - vor.points[p1] # tangent
            t /= np.linalg.norm(t)
            n = np.array([-t[1], t[0]])  # normal

            midpoint = vor.points[[p1, p2]].mean(axis=0)
            direction = np.sign(np.dot(midpoint - center, n)) * n
            far_point = vor.vertices[v2] + direction * radius

            new_region.append(len(new_vertices))
            new_vertices.append(far_point.tolist())
            

        # sort region counterclockwise
        vs = np.asarray([new_vertices[v] for v in new_region])
        c = vs.mean(axis=0)
        angles = np.arctan2(vs[:,1] - c[1], vs[:,0] - c[0])
        new_region = np.array(new_region)[np.argsort(angles)]

        # finish
        new_regions.append(new_region.tolist())

    return new_regions, np.asarray(new_vertices)

In [514]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import *
import os
import shutil
import glob
import scipy.spatial
import sys
from shapely.geometry import MultiPoint, Point, Polygon,LineString
from scipy.spatial import Voronoi


#Define the attributes of a grain: x,y,r,tesselation area and stress component s22:
class grain:
    def __init__(self, x_, y_, r_, dt_area,s22,m22):
        self.x = x_
        self.y = y_
        self.r = r_
        self.dt_area = dt_area
        self.s22 = s22
        self.m22 = m22

#Define the class and use the scipy.spatial.Voronoi to calculate the voronoi area 
#the input is the dataframe read from DEM*.dat
#the output is first the list of grain attributes and second the packing density
class Voronoi:
    
    def __init__(self,file):
        self.file = file
    
    def grains(self):
        from shapely.geometry import MultiPoint, Point, Polygon,LineString
        from scipy.spatial import Voronoi
        base=os.path.basename(self.file)
        output=os.path.splitext(base)[0]+".ps"
        output_grain = os.path.splitext(base)[0]+"_stress.dat"
        data = pd.read_csv(self.file,header= None, sep='\t')
        radii = data[1]
        nbgrains = len(radii)
        x = data[2]
        y = data[3]
        M22 = data[26] 

        points = np.vstack((x, y)).T
        vor = Voronoi(points)
        regions, vertices = voronoi_finite_polygons_2d(vor)
        # Build the boundary
        x_margin = data[2]+data[1]
        y_margin = data[3]+data[1]  
        y_max_left = 0
        x_max_bottom = 0
        for i in range(len(x_margin)):
            if x[i]<= radii.max(): 
                if y_max_left <= y_margin[i]: y_max_left = y_margin[i]
            if y[i]<= radii.max(): 
                if x_max_bottom <= x_margin[i]: x_max_bottom = x_margin[i]
        points_margin = np.vstack((x, y_margin)).T
        pts_bound = MultiPoint([[0,0],[x_max_bottom,0],[0,y_max_left]])
        pts_ini = MultiPoint([Point(i) for i in points_margin])
        pts = MultiPoint([p for p in pts_ini]+ [p for p in pts_bound])
        mask = pts.convex_hull
        
#         pts = MultiPoint([Point(i) for i in points])
#         mask = pts.convex_hull
        
        t_area = 0 # Tesselation area in total
        g_area = 0 # Grain area in total
        # creating list        
        grains = [] 
        
        f1 = open(output, "w")
         #Define the width and height of postscript
        wd = x.max()*12000
        ht = y.max()*12000 
        
        f1.write("%!PS-Adobe-3.0 EPSF-3.0 \n") 
        f1.write("%%BoundingBox:")
        f1.write("%.3f %.3f %.3f %.3f \n"% (-1,-1,1+wd,1+ht))       
        f1.write("%%Creator SpongeAmber \n")
        f1.write("%%Title: Voronoi Tessellation \n")

        for i in range(len(regions)):
#             if i==99:
            dt_area = 0 # Tesselation area per cell
            dg_area = 0 # Grain area per cell
            f1.write("newpath \n")
            polygon = vertices[regions[i]]
            shape = list(polygon.shape)
            shape[0] += 1
            p = Polygon(np.append(polygon, polygon[0]).reshape(*shape)).intersection(mask)
            poly = np.array(list(zip(p.boundary.coords.xy[0][:-1], p.boundary.coords.xy[1][:-1])))

            for j in range(len(poly)):
                if j == len(poly)-1: k = 0
                else: k = j+1
                next_x = poly[k][0]
                next_y = poly[k][1]
                current_x = poly[j][0]
                current_y = poly[j][1]
                f1.write("%.3f %.3f \n"% (10000*current_x,10000*current_y))
                if(j==0):
                    f1.write("moveto \n")
                else:
                    f1.write("lineto \n")
                dx = next_x - current_x
                dy = next_y - current_y
                edge = np.sqrt(dx*dx+dy*dy)
                #Discard stretchy cells whose edge is longer than 4 times the inside grain radius 
                if edge > (4*radii[i]):
                    dt_area=0 
                    dg_area = 0
                    break
                else:
                    dt_area = dt_area + (current_x * next_y - current_y * next_x)
                    dg_area =  math.pi * radii[i] * radii[i]

            #The tesselation area for each grain whose index is pt_index
            dt_area = math.fabs(dt_area)/2
            #Append the current grain attributes to the list grains[]
            if dt_area !=0: 
                grains.append(grain(x[i], y[i],radii[i],dt_area,M22[i]/dt_area,M22[i]))
            else: 
                grains.append(grain(x[i], y[i],radii[i],0.,0.,0.))
            g_area = g_area + dg_area
            t_area = t_area + dt_area
            f1.write("closepath \n")
            f1.write("1 setlinewidth 1 0 0 setrgbcolor \n")
            f1.write("stroke \n")
            f1.write("newpath %.3f %.3f %.3f"%(10000*x[i],10000*y[i],7000*radii[i]))
            f1.write(" 0.0 setlinewidth 0.4 setgray 0 360 arc gsave fill grestore \n")
        f1.close()

        #calculate the packing density
        void_area = t_area - g_area
        density = g_area / t_area
        
        # Output the stress to file
        f2 = open(output_grain, "w")
        f2.write("Grain x, y, Radius, Tesselation area, Stress 22, M22 \n")
        for g in grains:
            f2.write("%f %f %f %f %f %f\n"%(g.x,g.y,g.r,g.dt_area,g.s22,g.m22))
        f2.close()
        return [grains,density]

In [ ]:
## Normalized stress component s11 by the maximum initial s11 at base of the flowfront
#the aspect ratio and volume studied
condition = ["dry"]
volume= 2
a= "02"

#The initial time-step
ini_dat = "DEM000001.dat"


for c in condition:
    while volume <7:
        v=str(volume)        
         #path to the outpufiles and excutable files(current location);
        output_path = "/home/amber/Documents/2d-lbm-dem-analysis/2d-lbm-dem/Analysis/a"+a+"/code/contactforces/scipy.spatial.Voronoi/"
        
        #path to the inputfiles(initial DEM*.dat);
        input_path_ini = "/home/amber/Documents/2d-lbm-dem-analysis/2d-lbm-dem/Analysis/a02/"+c+"_v"+v+"00dat/"

        #path to the inputfiles(DEM*.dat at time-step corresponding to 3tau_c for different volumes);
        input_path_3tc= "/home/amber/Documents/2d-lbm-dem-analysis/2d-lbm-dem/Analysis/a"+a+"/code/contactforces/"
 
                
## Process the initial DEM*.dat and obtain the maximum initial s22 at base of flowfront:max_s22;
        # Copy the corresponding initial DEM*.dat to output_path;
        filename = input_path_ini+ini_dat
        os.chdir(output_path)
        file_ini = shutil.copyfile(filename,"./"+c+"-v"+v+"_ini.dat")
        
         #calculate density and grain attributes using class Voronoi
        data_ini = pd.read_csv(file_ini,header= None, sep='\t')
        ini = Voronoi(file_ini)
        ini_grains=ini.grains()[0]
        print ("The initial packing density of a="+a+" V="+v+"0000cm3 ={}".format(ini.grains()[1]))
        #determine the flowfront location
        max_radius = data_ini[1].max()
        max_x = data_ini[2].max()
        flowfront_x = max_x - 30*max_radius
        max_s22 = 0.
        #calculate max_s22
        for i in range(len(data_ini[1])):
            x_ = ini_grains[i].x
            y_ = ini_grains[i].y
            r_ = ini_grains[i].r
            s22_ = ini_grains[i].s22
            if y_ <= 2*r_ and x_>=flowfront_x:
                if s22_>max_s22:
                    max_s22=s22_
        print(max_s22)

## Process the 3tau_c DEM*.dat and obtain the normalized s22: nmlz_s22 at base of flowfront;
         # List of all the files at 3tauc_c;
        os.chdir(input_path_3tc)     
        files = filter(os.path.isfile, os.listdir('./')) 
        # Copy the corresponding DEM*.dat at 3 tau_c to output_path;
        for doc in files:
            if doc.startswith(c+"_v"+v):
                file_3tc = shutil.copyfile(doc,output_path+c+"-v"+v+"_3tc.dat")  
                
        os.chdir(output_path) 

        data_3tc = pd.read_csv(file_3tc,header= None, sep='\t')

        #determine the flowfront location
        max_radius = data_3tc[1].max()
        max_x = data_3tc[2].max()
        flowfront_x = max_x - 30*max_radius
        flowfront_l = 30*max_radius
        #calculate density and class grain using class Voronoi
        at_3tc = Voronoi(file_3tc)
        at_3tc_density = at_3tc.grains()[1]
        at_3tc_grains = at_3tc.grains()[0]
        print ("The packing density of a="+a+" V="+v+"0000cm3 at 3 tau_c={}".format(at_3tc_density))

        #Writing the stress compnent to txt
        output = c+"-v"+v+"base.txt"
        f2 = open(output, "w")
        for i in range(len(data_3tc[1])):
        # x_, y_, r_, dt_area,s22
         # x_, y_, r_, dt_area,s22
            x_ = at_3tc_grains[i].x
            y_ = at_3tc_grains[i].y
            r_ = at_3tc_grains[i].r
            s22_ = at_3tc_grains[i].s22
            if  y_ <= 2*r_ and x_>=flowfront_x:
                nmlz_x= (x_-flowfront_x)/flowfront_l
                if s22_!=0:         
                    nmlz_s22= s22_/max_s22
                    print(nmlz_x,nmlz_s22)
                    f2.write("%f %f \n"% (nmlz_x,nmlz_s22))   
                else:
                    f2.write("%f %f \n"% (nmlz_x,0))   
        f2.close()
        volume = volume +1

            
      

Exception ignored in: <function BaseGeometry.__del__ at 0x7f1dedca64d0>
Traceback (most recent call last):
  File "/home/amber/.local/lib/python3.7/site-packages/shapely/geometry/base.py", line 240, in __del__
    self.empty(val=None)
  File "/home/amber/.local/lib/python3.7/site-packages/shapely/geometry/base.py", line 227, in empty
    self._lgeos.GEOSGeom_destroy(self.__geom__)
KeyboardInterrupt: 


The initial packing density of a=02 V=20000cm3 =0.8315370507124328
532.6208326475497
The packing density of a=02 V=20000cm3 at 3 tau_c=0.8227008983704063
0.07238377161256092 2.2916667648611963e-05
0.999999999999999 2.409698746131528e-05
0.022086453963057407 0.021186975738630036
The initial packing density of a=02 V=30000cm3 =0.8350692853881787
1405.2810878773682
